In [32]:
from selenium import webdriver
import pandas as pd
import numpy as np
from selenium.webdriver.common.by import By
import random
from datetime import datetime, timedelta
import pickle
from fuzzywuzzy import fuzz, process
from fractions import Fraction
import time
import collections
import concurrent.futures
from collections import defaultdict
import json
import calendar
import plotly.express as px


In [2]:
def get_stats(url):
    driver = webdriver.Chrome()
    try:
        driver.implicitly_wait(random.randint(0, 2))
        driver.get(url)
        button = driver.find_elements(By.XPATH, '(//button[@class="css-12l9xvx-ExpandButton e7i6a0l0"])[2]')[0]
        div1 = driver.find_elements(By.XPATH, '//li[@class="css-radwzz-Stat e683amr6"]')
        t = {k[1]: [k[0], k[2]] for k in [i.split('\n') for i in [x for x in [d.text for d in div1]
              if x not in ['Shots','Passes','Expected goals (xG)','Discipline','Defence','Duels']]]}
        score = driver.find_element(By.XPATH, '//span[@class="css-bw7eig-topRow"]').get_attribute('textContent').split(' ')
        t['score'] = [score[0], score[2]]
        posession = driver.find_element(By.XPATH, '//div[@class="css-7s52se-PossessionWheel e683amr3"]').text.split('\n')
        t['posession'] = [posession[1].replace('%',''), posession[2].replace('%','')]
        league = driver.find_element(By.XPATH, '//span[@class="css-784syn-LeagueName e1fpx9th0"]').text
        game_stats = {}
        for name, stats in t.items():
            if '(' in ''.join(stats):
                s = stats[0].split(' ') + stats[1].split(' ')
                if len(s) == 4:
                    s = [i.replace('(', '').replace(')','').replace('%','') for i in s]
                    game_stats[name] = [s[0], s[2]]
                    game_stats[name + ' percentage'] = [int(s[1])/100, int(s[3])/100]
                else:
                    s = [i for i in s if '%' not in i]
                    game_stats[name] = s
            else:
                game_stats[name] = stats
        game_stats['league'] = league
        game_stats['url'] = url
    except:
        game_stats = []
    driver.quit()
    return game_stats

In [3]:
get_stats('https://www.fotmob.com/match/3093536/matchfacts/espanyol-vs-deportivo-alaves')

{'Total shots': ['19', '1'],
 'Big chances': ['5', '0'],
 'Big chances missed': ['3', '0'],
 'Accurate passes': ['451', '160'],
 'Accurate passes percentage': [0.83, 0.65],
 'Fouls committed': ['16', '19'],
 'Offsides': ['2', '0'],
 'Corners': ['2', '1'],
 'Shots off target': ['6', '1'],
 'Shots on target': ['9', '0'],
 'Blocked shots': ['4', '0'],
 'Hit woodwork': ['0', '0'],
 'Shots inside box': ['12', '1'],
 'Shots outside box': ['7', '0'],
 'Passes': ['546', '245'],
 'Own half': ['222', '80'],
 'Opposition half': ['229', '80'],
 'Accurate long balls': ['30', '21'],
 'Accurate long balls percentage': [0.5, 0.34],
 'Accurate crosses': ['7', '1'],
 'Accurate crosses percentage': [0.35, 0.11],
 'Throws': ['17', '20'],
 'Yellow cards': ['4', '3'],
 'Red cards': ['0', '1'],
 'Tackles won': ['15', '6'],
 'Tackles won percentage': [0.94, 0.46],
 'Interceptions': ['3', '8'],
 'Blocks': ['0', '0'],
 'Clearances': ['5', '19'],
 'Keeper saves': ['0', '7'],
 'Duels won': ['67', '54'],
 'Ground 

In [36]:
xg_leagues = {'UNITED STATES - MAJOR LEAGUE SOCCER':'world/usa/mls','ENGLAND - CHAMPIONSHIP':'english/championship',
              'ENGLAND - PREMIER LEAGUE': 'english/premier-league', 'GERMANY - BUNDESLIGA': 'germany/bundesliga',
              'FRANCE - LIGUE 1': 'france/ligue-1','AUSTRIA - BUNDESLIGA': 'austria/bundesliga',
              'PORTUGAL - LIGA PORTUGAL':'portugal/primeira-liga', 'BELGIUM - FIRST DIVISION A':'belgium/jupiler-pro-league',
              'SWITZERLAND - SUPER LEAGUE':'switzerland/super-league','SPAIN - LALIGA':'spain/la-liga-primera',
             'DENMARK - SUPERLIGAEN':'denmark/superligaen', 'MEXICO - LIGA MX APERTURA':'world/mexico/liga-mx',
             'NETHERLANDS - EREDIVISIE':'netherlands/eredivisie', 'SCOTLAND - PREMIERSHIP': 'scottish/premiership'}

In [4]:
def fotmob_game_data(url):
    try:
        driver = webdriver.Chrome()
        driver.implicitly_wait(3)
        driver.get(url)
        divs = driver.find_elements(By.XPATH, '//div[@class="css-aoqfcq-Group e7pc1841"]')
        links = []
        for count, bucket in enumerate(divs):
            league_name = bucket.find_elements(By.XPATH, '//a[@class="css-vfisw6-GroupTitleLink e7pc1842"]')[count].text
            if league_name not in xg_leagues.keys():
                continue
            url = bucket.find_elements(By.TAG_NAME, 'a')
            urls = [u.get_attribute('href') for u in url]
            urls = urls[1:]
            links.extend(urls)
    except:
        links = []
    driver.quit()
    return links

In [54]:
fotmob_game_data('https://www.fotmob.com/?date=20220814')

['https://www.fotmob.com/match/3787604/matchfacts/nashville-sc-vs-minnesota-united',
 'https://www.fotmob.com/match/3787587/matchfacts/seattle-sounders-fc-vs-real-salt-lake',
 'https://www.fotmob.com/match/3900949/matchfacts/nottingham-forest-vs-west-ham-united',
 'https://www.fotmob.com/match/3900946/matchfacts/chelsea-vs-tottenham-hotspur',
 'https://www.fotmob.com/match/3918012/matchfacts/cadiz-vs-real-sociedad',
 'https://www.fotmob.com/match/3918017/matchfacts/valencia-vs-girona',
 'https://www.fotmob.com/match/3918014/matchfacts/almeria-vs-real-madrid',
 'https://www.fotmob.com/match/3903556/matchfacts/mainz-05-vs-union-berlin',
 'https://www.fotmob.com/match/3903552/matchfacts/bayern-m%C3%BCnchen-vs-wolfsburg',
 'https://www.fotmob.com/match/3904402/matchfacts/lorient-vs-lyon',
 'https://www.fotmob.com/match/3904399/matchfacts/ac-ajaccio-vs-lens',
 'https://www.fotmob.com/match/3904400/matchfacts/auxerre-vs-angers',
 'https://www.fotmob.com/match/3904407/matchfacts/reims-vs-cler

In [55]:
d1 = datetime(2020, 1, 1)
d2 = datetime(2022, 8, 15)
days = [d1 + timedelta(days=x) for x in range((d2-d1).days + 1)]
days = [d.strftime('%Y%m%d') for d in days]

In [56]:
links = [f'https://www.fotmob.com/?date={day}' for day in days]

In [59]:
len(links)

958

In [61]:
game_links = []
for i in range(0, len(links), 5):
    chunk = links[i:i+5]
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = executor.map(fotmob_game_data, chunk)
    for result in results:
        game_links.append(result)
    print(f'{len(links) - i} links to go')


958 links to go
953 links to go
948 links to go
943 links to go
938 links to go
933 links to go
928 links to go
923 links to go
918 links to go
913 links to go
908 links to go
903 links to go
898 links to go
893 links to go
888 links to go
883 links to go
878 links to go
873 links to go
868 links to go
863 links to go
858 links to go
853 links to go
848 links to go
843 links to go
838 links to go
833 links to go
828 links to go
823 links to go
818 links to go
813 links to go
808 links to go
803 links to go
798 links to go
793 links to go
788 links to go
783 links to go
778 links to go
773 links to go
768 links to go
763 links to go
758 links to go
753 links to go
748 links to go
743 links to go
738 links to go
733 links to go
728 links to go
723 links to go
718 links to go
713 links to go
708 links to go
703 links to go
698 links to go
693 links to go
688 links to go
683 links to go
678 links to go
673 links to go
668 links to go
663 links to go
658 links to go
653 links to go
648 link

In [65]:
all_links = [i for j in game_links for i in j]

In [68]:
open_file = open(file_name, "wb")
pickle.dump(all_links, open_file)
open_file.close()

In [4]:
file_name = "all_links.pkl"

In [6]:
open_file = open(file_name, "rb")
game_links = pickle.load(open_file)
open_file.close()

In [7]:
len(game_links)

4540

In [75]:
open_file = open('stats_data.pkl', "wb")
pickle.dump(stats_data, open_file)
open_file.close()

In [82]:
len(stats_data)

4005

In [78]:
len(game_links)

4540

In [81]:
t = game_links[4001:4540]

In [83]:
for i in range(0, len(t), 3):
    chunk = t[i:i+3]
    try:
        with concurrent.futures.ThreadPoolExecutor() as executor:
            results = executor.map(get_stats, chunk)
        for result in results:
            stats_data.append(result)
        time.sleep(3)
    except:
        print(f'issue between {i} and {i + 3}')
        errors.append(chunk)
    print(f'{len(game_links) - i} links to go')

4540 links to go
4537 links to go
4534 links to go
4531 links to go
4528 links to go
4525 links to go
4522 links to go
4519 links to go
4516 links to go
4513 links to go
4510 links to go
4507 links to go
4504 links to go
4501 links to go
4498 links to go
4495 links to go
4492 links to go
4489 links to go
4486 links to go
4483 links to go
4480 links to go
4477 links to go
4474 links to go
4471 links to go
4468 links to go
4465 links to go
4462 links to go
4459 links to go
4456 links to go
4453 links to go
4450 links to go
4447 links to go
4444 links to go
4441 links to go
4438 links to go
4435 links to go
4432 links to go
4429 links to go
4426 links to go
4423 links to go
4420 links to go
4417 links to go
4414 links to go
4411 links to go
4408 links to go
4405 links to go
4402 links to go
4399 links to go
4396 links to go
4393 links to go
4390 links to go
4387 links to go
4384 links to go
4381 links to go
4378 links to go
4375 links to go
4372 links to go
4369 links to go
4366 links to 

In [84]:
len(stats_data)

4544

In [85]:
f = []
for dic in stats_data:
    d = {}
    if isinstance(dic, list):
        continue
    for k, v in dic.items():
        try:
            d[k] = float(v[0]) - float(v[1])
        except:
            d[k] = v
    f.append(d)


In [86]:
df = pd.DataFrame(f)

In [87]:
len(df)

4373

In [73]:
fig = px.scatter(df, x="Accurate passes percentage", y="Expected goals (xG)", color = 'league')
fig.show()


In [89]:
df.to_csv('fulltime.csv')